This Note Book is for Capstone project

import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

There are lot of accidents happening in Sietal City due to several reasons and there are many people injured and several fatalities during 2019. The idea is to provide some predictions on the accidents with severity based on the historical data, so that traveler may be careful while driving through those places and/or avoid such routes.  

In [2]:
import pylab as pl
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

There is a data set available with historical data contains the accidetal information with severity and various factors caused the accident. These attributes will help to Predict the future data. Imported the data set is available in csv file. Severity codes are 1 and 2, 2 being highly severe as there are injuries/Fatalities and 1 being not major as these are property damages only.

In [3]:
RaodAcc_df = pd.read_csv("C:/Users/jpot2560/Downloads/Data-Collisions.csv")
RaodAcc_df.head()

C:\Users\jpot2560\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [4]:
RaodAcc_df.shape

(194673, 38)

In [5]:
RaodAcc_df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


As the Data Contained Data Like Road condition,Lighting,Juntion,speed which caused the damage and impact. There is lot of additional data which is not required for prediction, I can remove irrelevant data for prediction.

In [6]:
Road_Acc_Clean = RaodAcc_df.drop(columns=['OBJECTID','INTKEY','LOCATION','SEVERITYCODE.1','SEVERITYDESC','LOCATION','INCKEY','X','Y','COLDETKEY','REPORTNO','STATUS','INTKEY','EXCEPTRSNCODE','EXCEPTRSNDESC','PERSONCOUNT','PEDCOUNT','PEDCYLCOUNT','VEHCOUNT','INCDATE','INCDTTM','SDOT_COLCODE','SDOT_COLDESC','SDOTCOLNUM','ST_COLDESC','HITPARKEDCAR'])

In [7]:
Road_Acc_Clean.shape

(194673, 14)

In [8]:
Road_Acc_Clean.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,SEGLANEKEY,CROSSWALKKEY
0,2,Intersection,Angles,At Intersection (intersection related),NaN,N,Overcast,Wet,Daylight,NaN,NaN,10,0,0
1,1,Block,Sideswipe,Mid-Block (not related to intersection),NaN,0,Raining,Wet,Dark - Street Lights On,NaN,NaN,11,0,0
2,1,Block,Parked Car,Mid-Block (not related to intersection),NaN,0,Overcast,Dry,Daylight,NaN,NaN,32,0,0
3,1,Block,Other,Mid-Block (not related to intersection),NaN,N,Clear,Dry,Daylight,NaN,NaN,23,0,0
4,2,Intersection,Angles,At Intersection (intersection related),NaN,0,Raining,Wet,Daylight,NaN,NaN,10,0,0


We have the Labelled data which should be classified into a category. Hence a supervised Machine Learning model like Logistic Regression to evaluate the Severity and probability which would help the travellers. Hence cleaning and processing the data to create logistic regression ML model.

In [10]:
Road_Acc_Clean['ADDRTYPE'].unique()

array(['Intersection', 'Block', 'Alley', nan], dtype=object)

In [11]:
Road_Acc_Clean = Road_Acc_Clean.replace(['Intersection', 'Block', 'Alley'],[1,2,3])

In [12]:
Road_Acc_Clean['COLLISIONTYPE'].unique()

array(['Angles', 'Sideswipe', 'Parked Car', 'Other', 'Cycles',
       'Rear Ended', 'Head On', nan, 'Left Turn', 'Pedestrian',
       'Right Turn'], dtype=object)

In [13]:
Road_Acc_Clean = Road_Acc_Clean.replace(['Angles','Sideswipe','Parked Car','Cycles','Rear Ended','Head On','Left Turn','Pedestrian',
       'Right Turn'],[1,2,3,4,5,6,7,8,9])

In [14]:
Road_Acc_Clean['JUNCTIONTYPE'].unique()

array(['At Intersection (intersection related)',
       'Mid-Block (not related to intersection)', 'Driveway Junction',
       'Mid-Block (but intersection related)',
       'At Intersection (but not related to intersection)', nan,
       'Unknown', 'Ramp Junction'], dtype=object)

In [15]:
Road_Acc_Clean = Road_Acc_Clean.replace(['At Intersection (intersection related)','Mid-Block (not related to intersection)','Driveway Junction','Mid-Block (but intersection related)','At Intersection (but not related to intersection)',
'Ramp Junction'],[1,2,3,4,5,6])

In [16]:
Road_Acc_Clean['INATTENTIONIND'].unique()

array([nan, 'Y'], dtype=object)

In [17]:
Road_Acc_Clean = Road_Acc_Clean.replace(['Y','N','Other','Unknown'],[20,21,22,23])

In [18]:
Road_Acc_Clean['SPEEDING'].unique()

array([nan, 20.])

In [19]:
Road_Acc_Clean['UNDERINFL'].unique()

array([21, '0', nan, '1', 20], dtype=object)

In [20]:
Road_Acc_Clean['WEATHER'].unique()

array(['Overcast', 'Raining', 'Clear', nan, 23, 22, 'Snowing',
       'Fog/Smog/Smoke', 'Sleet/Hail/Freezing Rain', 'Blowing Sand/Dirt',
       'Severe Crosswind', 'Partly Cloudy'], dtype=object)

In [21]:
Road_Acc_Clean = Road_Acc_Clean.replace(['Overcast','Raining','Clear','Snowing','Fog/Smog/Smoke','Sleet/Hail/Freezing Rain','Blowing Sand/Dirt','Severe Crosswind','Partly Cloudy'],[1,2,3,4,5,6,7,8,9])

In [22]:
Road_Acc_Clean['ROADCOND'].unique()

array(['Wet', 'Dry', nan, 23, 'Snow/Slush', 'Ice', 22, 'Sand/Mud/Dirt',
       'Standing Water', 'Oil'], dtype=object)

In [23]:
Road_Acc_Clean = Road_Acc_Clean.replace(['Wet','Dry','Snow/Slush','Ice','Sand/Mud/Dirt','Standing Water','Oil'],[1,2,3,4,5,6,7])

In [24]:
Road_Acc_Clean['LIGHTCOND'].unique()

array(['Daylight', 'Dark - Street Lights On', 'Dark - No Street Lights',
       nan, 23, 'Dusk', 'Dawn', 'Dark - Street Lights Off', 22,
       'Dark - Unknown Lighting'], dtype=object)

In [25]:
Road_Acc_Clean = Road_Acc_Clean.replace(['Daylight','Dark - Street Lights On','Dark - No Street Lights','Dusk','Dawn','Dark - Street Lights Off','Dark - Unknown Lighting'],[1,2,3,4,5,6,7])

In [26]:
Road_Acc_Clean = Road_Acc_Clean.fillna(-1)

In [27]:
Road_Acc_Clean.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,JUNCTIONTYPE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,SEGLANEKEY,CROSSWALKKEY
0,2,1.0,1.0,1.0,-1.0,21,1.0,1.0,1.0,-1.0,-1.0,10,0,0
1,1,2.0,2.0,2.0,-1.0,0,2.0,1.0,2.0,-1.0,-1.0,11,0,0
2,1,2.0,3.0,2.0,-1.0,0,1.0,2.0,1.0,-1.0,-1.0,32,0,0
3,1,2.0,22.0,2.0,-1.0,21,3.0,2.0,1.0,-1.0,-1.0,23,0,0
4,2,1.0,1.0,1.0,-1.0,0,2.0,1.0,1.0,-1.0,-1.0,10,0,0


In [28]:
Road_Acc_Clean.dtypes

SEVERITYCODE        int64
ADDRTYPE          float64
COLLISIONTYPE     float64
JUNCTIONTYPE      float64
INATTENTIONIND    float64
UNDERINFL          object
WEATHER           float64
ROADCOND          float64
LIGHTCOND         float64
PEDROWNOTGRNT     float64
SPEEDING          float64
ST_COLCODE         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
dtype: object

It is required to convert the Data types of all attributes to integers.

In [29]:
Road_Acc_Clean = Road_Acc_Clean[pd.to_numeric(Road_Acc_Clean['ST_COLCODE'], errors='coerce').notnull()]

Road_Acc_Clean['ST_COLCODE'] = Road_Acc_Clean['ST_COLCODE'].astype('int')

In [30]:
Road_Acc_Clean = Road_Acc_Clean[pd.to_numeric(Road_Acc_Clean['UNDERINFL'], errors='coerce').notnull()]

Road_Acc_Clean['UNDERINFL'] = Road_Acc_Clean['UNDERINFL'].astype('int')

In [31]:
Road_Acc_Clean = Road_Acc_Clean[pd.to_numeric(Road_Acc_Clean['ROADCOND'], errors='coerce').notnull()]

Road_Acc_Clean['ROADCOND'] = Road_Acc_Clean['ROADCOND'].astype('int')


In [32]:
Road_Acc_Clean = Road_Acc_Clean[pd.to_numeric(Road_Acc_Clean['ADDRTYPE'], errors='coerce').notnull()]

Road_Acc_Clean['ADDRTYPE'] = Road_Acc_Clean['ADDRTYPE'].astype('int')



In [33]:
Road_Acc_Clean = Road_Acc_Clean[pd.to_numeric(Road_Acc_Clean['COLLISIONTYPE'], errors='coerce').notnull()]

Road_Acc_Clean['COLLISIONTYPE'] = Road_Acc_Clean['COLLISIONTYPE'].astype('int')



In [34]:
Road_Acc_Clean = Road_Acc_Clean[pd.to_numeric(Road_Acc_Clean['WEATHER'], errors='coerce').notnull()]

Road_Acc_Clean['WEATHER'] = Road_Acc_Clean['WEATHER'].astype('int')


In [35]:
Road_Acc_Clean = Road_Acc_Clean[pd.to_numeric(Road_Acc_Clean['LIGHTCOND'], errors='coerce').notnull()]

Road_Acc_Clean['LIGHTCOND'] = Road_Acc_Clean['LIGHTCOND'].astype('int')

In [36]:
Road_Acc_Clean = Road_Acc_Clean[pd.to_numeric(Road_Acc_Clean['JUNCTIONTYPE'], errors='coerce').notnull()]

Road_Acc_Clean['JUNCTIONTYPE'] = Road_Acc_Clean['JUNCTIONTYPE'].astype('int')

In [37]:
Road_Acc_Clean = Road_Acc_Clean[pd.to_numeric(Road_Acc_Clean['INATTENTIONIND'], errors='coerce').notnull()]

Road_Acc_Clean['INATTENTIONIND'] = Road_Acc_Clean['INATTENTIONIND'].astype('int')

In [38]:
Road_Acc_Clean = Road_Acc_Clean[pd.to_numeric(Road_Acc_Clean['SPEEDING'], errors='coerce').notnull()]

Road_Acc_Clean['SPEEDING'] = Road_Acc_Clean['SPEEDING'].astype('int')

In [39]:
Road_Acc_Clean = Road_Acc_Clean[pd.to_numeric(Road_Acc_Clean['PEDROWNOTGRNT'], errors='coerce').notnull()]

Road_Acc_Clean['PEDROWNOTGRNT'] = Road_Acc_Clean['PEDROWNOTGRNT'].astype('int')

In [40]:
Road_Acc_Clean.dtypes

SEVERITYCODE      int64
ADDRTYPE          int32
COLLISIONTYPE     int32
JUNCTIONTYPE      int32
INATTENTIONIND    int32
UNDERINFL         int32
WEATHER           int32
ROADCOND          int32
LIGHTCOND         int32
PEDROWNOTGRNT     int32
SPEEDING          int32
ST_COLCODE        int32
SEGLANEKEY        int64
CROSSWALKKEY      int64
dtype: object

In [41]:
Road_Acc_Clean.shape

(189787, 14)

Creating the in dependant Data (X) and independant data(Y) from the dataframe.

In [44]:
Road_Acc_Clean_df = Road_Acc_Clean[['ADDRTYPE','COLLISIONTYPE','JUNCTIONTYPE','INATTENTIONIND','UNDERINFL','WEATHER','ROADCOND','LIGHTCOND','SPEEDING','PEDROWNOTGRNT','ST_COLCODE','SEGLANEKEY','CROSSWALKKEY']]
X = np.asarray(Road_Acc_Clean_df)
X[0:5]

array([[ 1,  1,  1, -1, 21,  1,  1,  1, -1, -1, 10,  0,  0],
       [ 2,  2,  2, -1,  0,  2,  1,  2, -1, -1, 11,  0,  0],
       [ 2,  3,  2, -1,  0,  1,  2,  1, -1, -1, 32,  0,  0],
       [ 2, 22,  2, -1, 21,  3,  2,  1, -1, -1, 23,  0,  0],
       [ 1,  1,  1, -1,  0,  2,  1,  1, -1, -1, 10,  0,  0]], dtype=int64)

In [45]:
Road_Acc_Clean_df.shape

(189787, 13)

In [46]:
Road_Acc_Clean_df.dtypes

ADDRTYPE          int32
COLLISIONTYPE     int32
JUNCTIONTYPE      int32
INATTENTIONIND    int32
UNDERINFL         int32
WEATHER           int32
ROADCOND          int32
LIGHTCOND         int32
SPEEDING          int32
PEDROWNOTGRNT     int32
ST_COLCODE        int32
SEGLANEKEY        int64
CROSSWALKKEY      int64
dtype: object

In [47]:
Road_Acc_Risk_df = Road_Acc_Clean[['SEVERITYCODE']]
y = np.asarray(Road_Acc_Risk_df['SEVERITYCODE'])
y [0:5]

array([2, 1, 1, 1, 2], dtype=int64)

In [48]:
y.shape

(189787,)

Importing the Train and test Split.

In [50]:
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Pre-processing the X Data(independant attributes)

In [52]:

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[-1.17746963, -0.76274945, -0.80758582, -0.43161881,  0.89930242,
        -0.5625006 , -0.42381026, -0.36127153, -0.22741933, -0.1587787 ,
        -0.88554489, -0.08156048, -0.13616807],
       [ 0.66150425, -0.60661631,  0.08241255, -0.43161881, -1.12187257,
        -0.38765242, -0.42381026, -0.18367928, -0.22741933, -0.1587787 ,
        -0.81666069, -0.08156048, -0.13616807],
       [ 0.66150425, -0.45048318,  0.08241255, -0.43161881, -1.12187257,
        -0.5625006 , -0.25105344, -0.36127153, -0.22741933, -0.1587787 ,
         0.62990748, -0.08156048, -0.13616807],
       [ 0.66150425,  2.51604639,  0.08241255, -0.43161881,  0.89930242,
        -0.21280425, -0.25105344, -0.36127153, -0.22741933, -0.1587787 ,
         0.00994969, -0.08156048, -0.13616807],
       [-1.17746963, -0.76274945, -0.80758582, -0.43161881, -1.12187257,
        -0.38765242, -0.42381026, -0.36127153, -0.22741933, -0.1587787 ,
        -0.88554489, -0.08156048, -0.13616807]])

Splitting the Data with Train Data 80% and Test Data 20%.

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=5)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (151829, 13) (151829,)
Test set: (37958, 13) (37958,)


Training the Model with liblinear optimizer available in the library.

In [55]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LR

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [64]:
yhat = LR.predict(X_test)
yhat.shape

(37958,)

Probability of Serverity 1 and Severity 2 accidents.

In [65]:
yhat_prob = LR.predict_proba(X_test)
yhat_prob

array([[0.77463438, 0.22536562],
       [0.80385311, 0.19614689],
       [0.63364574, 0.36635426],
       ...,
       [0.78943188, 0.21056812],
       [0.68689723, 0.31310277],
       [0.72667788, 0.27332212]])

Accuracy score using the jaccard similarity score and f1_score

In [73]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test,yhat)

0.7290953158754413

In [74]:
from sklearn.metrics import f1_score
f1_score(y_test, yhat, average='weighted') 

0.6576413153958554

In [75]:
print (classification_report(y_test, yhat))

              precision    recall  f1-score   support

           1       0.73      0.98      0.83     26538
           2       0.76      0.15      0.25     11420

    accuracy                           0.73     37958
   macro avg       0.74      0.56      0.54     37958
weighted avg       0.74      0.73      0.66     37958

